In [6]:
%pip install llama-index-llms-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-finetuning
%pip install llama-index-readers-file
%pip install llama_index
%pip install llama_index.llms.huggingface
%pip install sentencepiece
%pip install einops
%pip install modelscope
%pip install llama-index-embeddings-huggingface

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/c9/bb/524971a3f038701b8751b71f0ff825a8df82c0f511d3860cb58a8fe93045/llama_index_core-0.10.68.post1-py3-none-any.whl (1.6 MB)
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/dd/60/54155af74479199bdbd34394af6d907be7103b8bb7990ec6ef6e54b7dd17/llama_index_llms_openai-0.1.31-py3-none-any.whl (12 kB)
ERROR: llama-parse 0.5.1 has requirement llama-index-core>=0.11.0, but you'll have llama-index-core 0.10.68.post1 which is incompatible.
ERROR: llama-index 0.11.2 has requirement llama-index-core<0.12.0,>=0.11.2, but you'll have llama-index-core 0.10.68.post1 which is incompatible.
ERROR: llama-index 0.11.2 has requireme

In [1]:
import json

from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.schema import MetadataMode

In [2]:
TRAIN_FILES = ["./data/train.txt"]
VAL_FILES = ["./data/val.txt"]

# TRAIN_CORPUS_FPATH = "./data/train_corpus.json"
# VAL_CORPUS_FPATH = "./data/val_corpus.json"

In [3]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f"Loaded {len(docs)} docs")

    parser = SentenceSplitter()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f"Parsed {len(nodes)} nodes")

    return nodes

In [4]:
train_nodes = load_corpus(TRAIN_FILES, verbose=True)
val_nodes = load_corpus(VAL_FILES, verbose=True)

Loading files ['./data/train.txt']
Loaded 1 docs


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 243.54it/s]


Parsed 5 nodes
Loading files ['./data/val.txt']
Loaded 1 docs


Parsing nodes: 100%|██████████| 1/1 [00:00<00:00, 122.25it/s]

Parsed 8 nodes


In [4]:
from llama_index.finetuning import generate_qa_embedding_pairs
from llama_index.core.evaluation import EmbeddingQAFinetuneDataset

/usr/local/lib/python3.8/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_adapter_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [13]:
# import os
# OPENAI_BASE_URL="https://api.wlai.vip/v1"
# OPENAI_API_KEY = "sk-"
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
# os.environ["OPENAI_BASE_URL"] = OPENAI_BASE_URL

In [14]:
# from llama_index.llms.openai import OpenAI


# train_dataset = generate_qa_embedding_pairs(llm=OpenAI(model="gpt-3.5-turbo",api_key=OPENAI_API_KEY,api_base=OPENAI_BASE_URL), nodes=train_nodes)
# val_dataset = generate_qa_embedding_pairs(llm=OpenAI(model="gpt-3.5-turbo",api_key=OPENAI_API_KEY,api_base=OPENAI_BASE_URL), nodes=val_nodes)

# train_dataset.save_json("train_dataset.json")
# val_dataset.save_json("val_dataset.json")

  0%|          | 1/344 [04:17<24:30:35, 257.25s/it]


KeyboardInterrupt: 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.llms.huggingface import HuggingFaceLLM

# 使用本地路径模型
model_name_or_path = "/share/new_models/Shanghai_AI_Laboratory/internlm2-chat-1_8b"
# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
llm = HuggingFaceLLM(model_name=model_name_or_path, tokenizer_name=model_name_or_path)

train_dataset = generate_qa_embedding_pairs(llm=llm, nodes=train_nodes)
val_dataset = generate_qa_embedding_pairs(llm=llm, nodes=val_nodes)

train_dataset.save_json("./data/train_dataset.json")
val_dataset.save_json("./data/val_dataset.json")


/usr/local/lib/python3.8/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_kwargs" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


The repository for /share/new_models/Shanghai_AI_Laboratory/internlm2-chat-1_8b contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co//share/new_models/Shanghai_AI_Laboratory/internlm2-chat-1_8b.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [5]:
# [Optional] Load
train_dataset = EmbeddingQAFinetuneDataset.from_json("./data/train_dataset.json")
val_dataset = EmbeddingQAFinetuneDataset.from_json("./data/train_dataset.json")

In [6]:
from llama_index.finetuning import SentenceTransformersFinetuneEngine

In [7]:
from modelscope import snapshot_download
# 第一个参数表示下载模型的型号，第二个参数是下载后存放的缓存地址，第三个表示版本号，默认 master
model_dir = snapshot_download('AI-ModelScope/bge-small-zh-v1.5', cache_dir='/root', revision='master')

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-08-29 22:46:53,823 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


In [7]:
finetune_engine = SentenceTransformersFinetuneEngine(
    train_dataset,
    model_id="/root/AI-ModelScope/bge-small-zh-v1___5",
    model_output_path="test_model",
    val_dataset=val_dataset,
)

In [8]:
finetune_engine.finetune()

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
embed_model = finetune_engine.get_finetuned_model()

In [10]:
embed_model

HuggingFaceEmbedding(model_name='test_model', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7f4ff097b5b0>, num_workers=None, max_length=512, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [11]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.schema import TextNode
from tqdm.notebook import tqdm
import pandas as pd

In [12]:
def evaluate(
    dataset,
    embed_model,
    top_k=5,
    verbose=False,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(
        nodes, embed_model=embed_model, show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            "is_hit": is_hit,
            "retrieved": retrieved_ids,
            "expected": expected_id,
            "query": query_id,
        }
        eval_results.append(eval_result)
    return eval_results

In [13]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer
from pathlib import Path


def evaluate_st(
    dataset,
    model_id,
    name,
):
    corpus = dataset.corpus
    queries = dataset.queries
    relevant_docs = dataset.relevant_docs

    evaluator = InformationRetrievalEvaluator(
        queries, corpus, relevant_docs, name=name
    )
    model = SentenceTransformer(model_id)
    output_path = "results/"
    Path(output_path).mkdir(exist_ok=True, parents=True)
    return evaluator(model, output_path=output_path)

In [14]:
%pip install ipywidgets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.


In [15]:
bge = "local:/root/AI-ModelScope/bge-small-zh-v1___5"
bge_val_results = evaluate(val_dataset, bge)

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
df_bge = pd.DataFrame(bge_val_results)

In [18]:
df_bge

,is_hit,retrieved,expected,query
0,True,"[990770fc-825d-455e-bcc5-bc859a4a355a, 2672652...",3f2beccc-723d-44a2-bb0d-e59c493b569d,65475173-f069-431e-9cfa-f4e0d0f034c8
1,True,"[2672652e-58ff-4280-9941-24f6cd00721a, 3f2becc...",3f2beccc-723d-44a2-bb0d-e59c493b569d,18c1aae5-e5a4-466c-9dea-cfa6ae9c4cb4
2,True,"[8d7405e8-ce84-4ea5-88e4-8a71cfda31e2, 990770f...",8d7405e8-ce84-4ea5-88e4-8a71cfda31e2,badad0ec-2045-4724-bc96-468154275042
3,True,"[2672652e-58ff-4280-9941-24f6cd00721a, 3f2becc...",8d7405e8-ce84-4ea5-88e4-8a71cfda31e2,c34f9bbf-7254-4984-bdf6-8d24e63a7c86
4,True,"[3f2beccc-723d-44a2-bb0d-e59c493b569d, 2672652...",80402cdd-a01d-414a-a9d8-db80b3be63f9,e1d911ea-bddf-4f4d-89bc-33fa9fd4fc55
5,True,"[2672652e-58ff-4280-9941-24f6cd00721a, 3f2becc...",80402cdd-a01d-414a-a9d8-db80b3be63f9,70ef2734-c540-43d0-9740-1989089006ca
6,True,"[8d7405e8-ce84-4ea5-88e4-8a71cfda31e2, 990770f...",2672652e-58ff-4280-9941-24f6cd00721a,fd9e3749-ea9f-43c7-a67e-5f1b049a2c41
7,True,"[2672652e-58ff-4280-9941-24f6cd00721a, 8d7405e...",2672652e-58ff-4280-9941-24f6cd00721a,2d4dff8c-f880-40f5-96a3-d8d3c470b068
8,True,"[2672652e-58ff-4280-9941-24f6cd00721a, 3f2becc...",990770fc-825d-455e-bcc5-bc859a4a355a,04099bd3-716b-4ce6-9d05-330406981c1c
9,True,"[3f2beccc-723d-44a2-bb0d-e59c493b569d, 2672652...",990770fc-825d-455e-bcc5-bc859a4a355a,5caceebb-ca9e-49d4-8cbf-00413a0d61c6


In [19]:
hit_rate_bge = df_bge["is_hit"].mean()
hit_rate_bge

1.0

In [20]:
evaluate_st(val_dataset, "/root/AI-ModelScope/bge-small-zh-v1___5", name="bge")

0.43166666666666664

In [21]:
finetuned = "local:test_model"
val_results_finetuned = evaluate(val_dataset, finetuned)

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [22]:
df_finetuned = pd.DataFrame(val_results_finetuned)

In [23]:
hit_rate_finetuned = df_finetuned["is_hit"].mean()
hit_rate_finetuned

1.0

In [ ]:
evaluate_st(val_dataset, "test_model", name="finetuned")